# Game plan
### Analyze transaction descriptions. How?:
- **Identify which company/person was the amount paid to**:
  - Identify categories of transaction types based on payment method.
  - Find out the frequency of these categories and use the most frequently used data.

- **Identify which category the companies fall into**:
  - APIs/LLMs (?):
    - Restaurant, Grocery, Clothing, Sports, Travel, Misc.



In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys, os

# add relevant folders to sys path (only needed for jupyter)
proj_path = Path().cwd()
for folder in next(os.walk(proj_path))[1]:
    if '.' not in folder: # ignore hidden folders like .git
        path_to_add = os.path.join(proj_path,folder)
        sys.path.append(path_to_add)

import data_functions

In [16]:
transactions = data_functions.load_transaction_data()
# transactions.head() mute for security reasons

In [17]:
def make_identifier(description):

    """Identify transaction type using method of payment. 
    The first two string elements in the description indicate type of transaction, 
    the payment method, or the institution facilitating the transaction (SEPA or BEA for instance)
    
    """
    elements = description.split()[:2]
    if ',' in elements[0]:
        elements[0] = elements[0].split(',')[0]

    identfier = elements[0] + '_' + elements[1]

    return identfier



In [18]:
identifiers = []

# append identifiers from transaction descriptions
for descriptions in transactions['description']:
    
    identifiers.append(make_identifier(descriptions))

transactions['identifier'] = identifiers

# print transaction table again
#transactions.head()

In [19]:
# frequency of different transaction type
#print(transactions['identifier'].explode().value_counts())